In [4]:
import pandas
from pandas import *
import numpy
from datetime import datetime

# Get the training data
df = pandas.read_csv("data/train.csv.gz")

# Reading test data set
df_test = pandas.read_csv("data/test.csv.gz")


display(df.sample(10))

,date,store,item,sales
877385,2015-06-25,1,49,32
44719,2015-06-15,5,3,19
813585,2015-10-13,6,45,66
60377,2013-04-30,4,4,24
125302,2016-02-09,9,7,46
862763,2015-06-11,3,48,81
512280,2015-09-28,1,29,51
124187,2013-01-20,9,7,39
733270,2015-11-11,2,41,24
431976,2015-11-07,7,24,42


In [3]:
# Add all variables that we would like to this model, as boolean 

# convert day of week into 3 booleans, so that hidden layers of neural network do the rest
def day_of_week_and_month(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    week_day = datetime_object.weekday()
    dow_0 = week_day % 2
    dow_1 = int(week_day / 2) % 2
    dow_2 = int(week_day / 4) % 2
    
    # Same for month
    month = datetime_object.month
    month_0 = month % 2
    month_1 = int(month / 2) % 2
    month_2 = int(month / 4) % 2
    month_3 = int(month / 8) % 2
    
    # TODO, same for calendar week + year 
    return (dow_0, dow_1, dow_2, month_0, month_1, month_2, month_3)

# Same for the store
def store_type(store):
    d_0 = store % 2
    d_1 = int(store / 2) % 2
    d_2 = int(store / 4) % 2
    d_3 = int(store / 8) % 2
    return (d_0, d_1, d_2, d_3)


# And for the item as well
def item_type(item):
    d_0 = item % 2
    d_1 = int(item / 2) % 2
    d_2 = int(item / 4) % 2
    d_3 = int(item / 8) % 2
    d_4 = int(item / 16) % 2
    d_5 = int(item / 32) % 2
    return (d_0, d_1, d_2, d_3, d_4, d_5)


def extrapolate_boolean(df):
    df['store_0'], df['store_1'], df['store_2'], df['store_3'] = zip(*df["store"].map(store_type))
    df['item_0'], df['item_1'], df['item_2'], df['item_3'], df['item_4'], df['item_5'] = zip(*df["item"].map(item_type))
    df['dow_0'], df['dow_1'], df['dow_2'], df['month_0'], df['month_1'], df['month_2'], df['month_3'] = zip(*df["date"].map(day_of_week_and_month))

    # Columns not needed anymore
    df = df.drop(['item', 'store'], axis=1)
    df = df.drop(['date'], axis=1)
    return df

#df = extrapolate_boolean(df)
#df_test = extrapolate_boolean(df_test)
# Column not needed anymore
df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [3]:
# Get maximum number of sale, adding 30%
#max = df['sales'].max()
#theory_max = int(max * 1.3)
#print("Maximum value is {0}, assuming maximum of {1}".format(max, theory_max))

# Should we do it like that?
#df['sales'] = df['sales'] / theory_max
# Remove result
#df = df.drop(['sales'], axis=1)
#df.head(5)

In [4]:
# Generate our training/validation datasets
from sklearn import model_selection

array = df.values

X = array[:,1:]
Y = array[:,0]
Y = Y.astype('int')

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [5]:
#Y_train = Y_train.astype('int')
#Y_validation = Y_validation.astype('int')

print(X_train)
print(Y_train)

[[0 1 0 ..., 0 0 1]
 [1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 1 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 1 ..., 0 1 0]
 [0 0 1 ..., 0 0 1]]
[89 22 83 ..., 38 99 26]


In [ ]:
from sklearn.neural_network import MLPClassifier


clf = MLPClassifier(alpha=1e-5,
                     hidden_layer_sizes=(15,), random_state=1)

# Train the model
clf.fit(X_train, Y_train)

In [ ]:
X_res = clf.predict(X_train)
print(X_res)
print(Y_train)

In [ ]:
# Prepare test data with the previous function

test_array = df_test.values
X_test = test_array[:,1:]
Y_test = clf.predict(X_test)
print("Result should be {0}".format(Y_test))
print(len(Y_test))
print(len(X_test))

In [ ]:
# Create new result dataframe:
result_df = df_test['id']
df_test['sales'] = Y_test
df_test.head(5)
df_result = df_test[['id', 'sales']]

# Write the dataframe as output
df.to_csv()
df_result.head(5)

df_result.to_csv("result.csv", index=False)